# How to Solve Sudoku with PuLP 
Optimization problem based on graph coloring solving a 3x3 Sudoku puzzle (as shown below).

**The 3x3 Sudoku Problem:**
<img src="files/images/prob.png" width="400">

First, let's talk about a little about Graph Coloring Problem. Essentially what it a graph is a collection of nodes with their associated edges. It could a directed or undirected graph (in this case, directionality of nodes are irrelevant, hence a Sudoku problem is an `undirected graph coloring problem`).

So, in order to model this as an optimization problem, we have to allocate the two most important ingredients of a graph (nodes and edges) in the context of Sudoku.
- In this case, there are a total of 9 numbers we can use to fill out the Sudoku cells. Hence the amoount *"colors"* we have in this case is exactly 9.
- Every cell in Sudoku is a **node**. For this problem, there are 81 "cells" or nodes in total. Each node is labeled with a number ranging from 1 to 81 (as shown below). 

- Edges are relationships between nodes. In the case of Sudoku, we have to model the relationship of column-wise nodes, row-wise nodes and box-wise nodes in order to observe the rules of Sudoku. So, in essence, an edge in this case is a pair-wise relationship between nodes.

## Optimisation Problem Setup

_**Parameters:**_<br>
<img src="files/images/param.png" width="450">





_**Objective function:** $$\min \sum_{k} y_{k}$$<br>
To minimize the amount of colors used. But due to the fact that we know exactly that we need 9 "colors" to color our Sudoku, this equation is redundant._
<br> <br> <br>

_**Constraint 1:**_ $$\sum_{k} x_{i k}=1,     \quad \forall i=[1,2,3 \dots 81]$$ 
<br>Each node can only be assigned one color k. Since we have binary variables for each node-colour combination, the sum over all colours for a single node needs to equal to one.
<br> <br> <br>

_**Constraint 2:**_
$$
x_{i k} \leq y_{k} \quad, \quad \forall i \in \text {Nodes}, \quad \forall k \in C
$$<br>
This constraint is to ensure that before node $i$ is assigned colour k, colour k must exist. Colour k exists only exists if and only if $y_k$ = 1. $x_{ik}$ cannot be 1 when $y_k$ = 0 therefore $y_k$ acts like a switch that signifies the existence of colour k. C is the set of colours. <br> <br> <br>

_**Constraint 3:**_
<br>$$\sum_{i} \sum_{j \atop j \neq i} x_{i k}+x_{j k} \leq 1, \quad \forall i, j \text { where } i \text { and } j \text { are connected by edges, } \forall k \in[1,2,3 \dots n]$$<br>
This undirected graph has no isolated nodes. We need to constraint that no adjacent nodes ($i$ and $j$) connected by an edge are not
assigned the same colour $k$. The above constraint equation will be able to do the aforementioned constraints. C is the set of colours. <br> <br> <br>

_**Constraint 4:**_<br>$$x_{i k}=1 \text { for all Cell $i$ that has a predefined value of } k$$<br>

For this Sudoku problem presented, there are 22 known values for nodes. So, our model should take into account these extra 22 constraints. 

*For example, looking at Node 1, which has a number "6" in it. So the extra constraint that has to be added for this particular node would be:* $$x_{16} = 1$$
<br> <br> <br>

_**Constraint 5:**_<br>
$$x_{i k} \text { and } y_{k} \text { are binary variables for all values of i and } k$$ 
<br> <br> <br>

**Completed 3x3 Sudoku Problem:**
<img src="files/images/sol.png" width="300">

## Code to solve the Sudoku Problem

Functions to aid in adding edges to the nodes:<br>

In [28]:
# This function creates pairs of relationships (edges) for every node

## This function creates edges for nodes inside a box

def edge_in_box(top_left_node, cube_length):
# This function creates edges for nodes inside a box.
# A box are collectively 9 nodes in a 3-by-3 matrix
    ed=[]
    x1=[a for a in range(top_left_node,top_left_node+cube_length)]
    x2 = [a+cube_length**2 for a in range(top_left_node,top_left_node+cube_length)]
    x3 = [a+2*cube_length**2 for a in range(top_left_node,top_left_node+cube_length)]
    full = x1 + x2 +x3    
    for a in full:
        for b in full:
            if a!=b:
                ed.append(tuple(sorted([a,b])))
    return(ed)

def edge_in_row(node_num,cube_length):    
# This function creates edges for nodes in the same row
    ed=[]
    full=[]
    x=int((node_num-0.1)//cube_length**2)
    start=(x*(cube_length**2)+1)
    end=(x*(cube_length**2)+1)+cube_length**2 
    for a in range(start,end):
        full.append(a)
    for a in full:
        for b in full:
            if a!=b:
                ed.append(tuple(sorted([a,b])))
    return(ed)   

def edge_in_col(node_num,cube_length):    
#This function creates edges for nodes in the same column
    ed=[]
    full=[]
    start = node_num%cube_length**2 
    end = cube_length**4 + 1
    step = cube_length**2
    if start == 0:
        start = 9   
    for x in range(start,end,step):
        full.append(x)
    for a in full:
        for b in full:
            if a!=b:
                ed.append(tuple(sorted([a,b])))
    return(ed)    

Additional functions to aid in solution presentation:

In [30]:
inf = 1e70
fmt = "{:3.0f}"
def is_valid(x):
    return x is not None and x > -inf and x < inf
def to_print(name, x, short, extra=""):
    if is_valid(x) and (not short or x != 0):
        print(name, fmt.format(x), "" if short else extra)
def print_sol(mod, short=True):
    print("Objective:", pulp.value(mod.objective))
    for (vname, v) in sorted(mod.variablesDict().items()):
        to_print(vname, v.value(), short)    

Setting up the edges:

In [44]:
import numpy as np
from pulp import *

## make the "edge" list for Sudoku 3x3, each element in list is a tuple
cube_length = 3      ## TO CHANGE!!!!
list_top_left_node = [1,4,7,28,31,34,55,58,61] ## TO CHANGE!!!!

edge=[]
for top_left_node in list_top_left_node:
    edge.extend(edge_in_box(top_left_node, cube_length))
for node_num in range(1,1+cube_length**4):
    edge.extend(edge_in_row(node_num,cube_length))
    edge.extend(edge_in_col(node_num,cube_length))  
edge = list(set(edge))

Some initial setup:

In [32]:
# A list of strings from "1" to "9" is created
seq_value = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]      
nodes = range(1,1+cube_length**4)
node_temp = []
node_temp=[node_temp.append(str(x)) for x in nodes]
colors = seq_value

### Putting everything together and initialising the LP problem:

In [34]:
# The sudoku3x3 problem is initialised
sudoku3x3 = LpProblem("Sudoku 3x3 Problem", LpMinimize) 

# The variables are created
x = LpVariable.dicts("x",(colors,nodes),0,1,LpInteger)    
y = LpVariable.dicts("y",colors,0,1,LpInteger)    

# OBJECTIVE FUNCTION
sudoku3x3 += sum(y[k] for k in colors) 
 
    
## Constraint 1: One cell can only have one value
for i in nodes:
        sudoku3x3 += sum([x[k][i] for k in colors]) == 1

# Constraint 2: Color k must exist before node i can be assigned color k.
### y_k >= x_ik 
for k in colors:
    for i in nodes:
        sudoku3x3 += y[k] >= x[k][i]

# Constraint 3: Nodes i and j (connected by edge) cannot have the same colour k
for k in colors:
    for (i,j) in edge:
        sudoku3x3 += x[k][i]+x[k][j] <= 1 

In [35]:
# Constraint 4: 22 constraints that incorporates the pre-defined values for certain nodes
sudoku3x3 += x["6"][1] == 1
sudoku3x3 += x["9"][10] == 1
sudoku3x3 += x["2"][28] == 1
sudoku3x3 += x["8"][73] == 1
sudoku3x3 += x["8"][47] == 1
sudoku3x3 += x["9"][48] == 1
sudoku3x3 += x["7"][20] == 1
sudoku3x3 += x["8"][6] == 1
sudoku3x3 += x["9"][7] == 1
sudoku3x3 += x["4"][8] == 1
sudoku3x3 += x["6"][15] == 1
sudoku3x3 += x["1"][16] == 1
sudoku3x3 += x["4"][23] == 1
sudoku3x3 += x["6"][31] == 1
sudoku3x3 += x["1"][32] == 1
sudoku3x3 += x["2"][43] == 1
sudoku3x3 += x["2"][51] == 1
sudoku3x3 += x["6"][59] == 1
sudoku3x3 += x["5"][63] == 1
sudoku3x3 += x["3"][71] == 1
sudoku3x3 += x["6"][79] == 1
sudoku3x3 += x["1"][78] == 1

In [39]:
pulp.LpStatus[sudoku3x3.status]

'Optimal'

In [40]:
print_sol(sudoku3x3)

Objective: 9.0
x_1_16   1 
x_1_21   1 
x_1_32   1 
x_1_4   1 
x_1_45   1 
x_1_46   1 
x_1_62   1 
x_1_65   1 
x_1_78   1 
x_2_14   1 
x_2_2   1 
x_2_27   1 
x_2_28   1 
x_2_43   1 
x_2_51   1 
x_2_57   1 
x_2_67   1 
x_2_80   1 
x_3_13   1 
x_3_19   1 
x_3_34   1 
x_3_39   1 
x_3_50   1 
x_3_60   1 
x_3_71   1 
x_3_74   1 
x_3_9   1 
x_4_11   1 
x_4_23   1 
x_4_36   1 
x_4_37   1 
x_4_49   1 
x_4_61   1 
x_4_69   1 
x_4_75   1 
x_4_8   1 
x_5_17   1 
x_5_24   1 
x_5_29   1 
x_5_3   1 
x_5_40   1 
x_5_52   1 
x_5_63   1 
x_5_64   1 
x_5_77   1 
x_6_1   1 
x_6_15   1 
x_6_26   1 
x_6_31   1 
x_6_38   1 
x_6_54   1 
x_6_59   1 
x_6_66   1 
x_6_79   1 
x_7_18   1 
x_7_20   1 
x_7_30   1 
x_7_42   1 
x_7_5   1 
x_7_53   1 
x_7_55   1 
x_7_70   1 
x_7_76   1 
x_8_12   1 
x_8_25   1 
x_8_35   1 
x_8_41   1 
x_8_47   1 
x_8_58   1 
x_8_6   1 
x_8_72   1 
x_8_73   1 
x_9_10   1 
x_9_22   1 
x_9_33   1 
x_9_44   1 
x_9_48   1 
x_9_56   1 
x_9_68   1 
x_9_7   1 
x_9_81   1 
y_1   1 
y_2   1 
y_3 

## Conclusion

And that's it! We have our solution for our Sudoku problem by modelling it as a graph coloring problem and modelling it as a LP problem.<br>

Just to re-iterate in regards to reading the solution. 

<br>If we take `x_9_81` as an example:
- "81" refers to which node
- "9" refers to what "color" or "number" is in that cell

So, `x_9_81` means Node 81 has the number "9" in it. And this corresponds to the image of the completed Sudoku problem provided.

**Completed 3x3 Sudoku Problem:**
<img src="files/images/sol.png" width="300">